# LAMMPS Tutorials 01. Running your first LAMMPS simulation!

### Author: Mark Tschopp, mark.a.tschopp.civ at mail.mil

Please contact me if you have a problem with this tutorial, so I can modify in Github.  I have added FAQs, and will update my versions of LAMMPS in the future to keep the scripts current.

The latest version of this [Jupyter Notebook](http://ipython.org/notebook.html) tutorial is available at https://github.com/mrkllntschpp/lammps-tutorials.

The original tutorials are given here: https://icme.hpc.msstate.edu/mediawiki/index.php/LAMMPS_tutorials.  A number of these tutorials are out of date and have been ported over into the current iPython Jupyter Notebook tutorials on github.

***

## Description:
<a id="Sec1"></a>
This is a quick tutorial to running a LAMMPS simulation on a Windows machine. For this simple example, the molecular simulation calculates the equilibrium lattice constant and corresponding cohesive energy for aluminum in the face-centered cubic (fcc) lattice structure (see below).

<img src="https://icme.hpc.msstate.edu/mediawiki/images/e/ef/Fcc_stereo.gif" alt="Face-centered Cubic Lattice Structure" title="FCC Lattice Structure" />

***
## Step 1: Download an Input File
<a id="Step1"></a>

We will use `pylammpsmpi` to run lammps calculations. `pylammpsmpi` will allow you run calculations directly from a jupyter notebook, and even submit the job on computing clusters. We start by importing and creating an object.

In [1]:
from pylammpsmpi import LammpsLibrary

In [12]:
lmp = LammpsLibrary(cores=1)

In `pylammpsmpi`, the commands are the same as in a conventional lammps script. The main commands such as `units`, `dimension` etc. below can be accessed as the member functions of `LammpsLibrary` object.

In [13]:
lmp.units("metal") 
lmp.dimension(3) 
lmp.boundary("p p p") 
lmp.atom_style("atomic") 

This section initializes the simulations. The `units` command specifies the units that will be used for the remainder of the simulation; `metal` uses Angstroms and eV, among other units. The `dimension 3` command specifies a 3-dimensional simulation cell will be used (2-D is also an option). The `boundary p p p` specifies periodic boundaries in the x-, y-, and z-directions. 

In [14]:
lmp.lattice("fcc", 4, "orient x", 1, 1, 0, "orient y", -1, 1, 0, "orient z", 0, 0, 1)  
lmp.region("box block", 0, 1, 0, 1, 0, 1, "units lattice")
lmp.create_box(1, "box")
lmp.create_atoms(1, "box")
lmp.replicate(1, 1, 1)

The `lattice` command specifies what type of lattice is used (fcc, bcc, hcp, etc.) and the number following this specifies the lattice constant. The `region` command specifies the simulation cell. Here, we have used lattice units and specified that the simulation cell box is to be 1 lattice unit in each direction. The `create_box` command following this will use the parameters outlined in the `region` command to actually create the box. The `replicate` command can be used to replicate the periodic cell in each direction. 

In [15]:
lmp.pair_style("eam/alloy") 
lmp.pair_coeff(" * * Al99.eam.alloy Al")
lmp.neighbor(2.0, "bin") 
lmp.neigh_modify("delay 10 check yes") 

The `pair_style` command specifies what kind of interatomic potential will be used, while the `pair_coeff` specifies the file that the pair potential coefficients are stored in. The file extension for the interatomic potential file can sometimes give a clue as to which format is being used (eam.alloy = eam/alloy).

In [16]:
lmp.compute("eng all pe/atom") 
lmp.compute("eatoms all reduce sum c_eng") 

Here, two computes are defined. In the first `compute` command, a variable named `eng` is defined to store the potential energy for each atom. In the second `compute` command, a variable named `eatoms` is defined to store the sum of all `eng` values. This is just to show how to use computes with user-defined variables.  Notice that the `pe` energy column during minimization is equivalent to the `c_eatoms` column, as expected.

In [17]:
lmp.reset_timestep(0) 
lmp.fix(1, "all box/relax iso", 0.0, "vmax", 0.001)
lmp.thermo(10) 
lmp.thermo_style("custom step pe lx ly lz press c_eatoms") 
lmp.min_style("cg") 
lmp.minimize(1e-5, 1e-5, 5000, 10000) 

The `reset_timestep` does just that. The `fix` command uses the `box/relax` setting, whereby all directions (`iso`) are relaxed to 0.0 Pa pressure for all atoms (`all`). The `thermo` specifies the output during minimization. The `thermo_style` specifies what type of output is shown to screen. Here I have used a `custom` list of metrics, including the time`step`, the potential energy (`pe`), the length of the box in x, y, and z (`lx`, `ly`, `lz`, respectively), the `press`ure, and the computed variable `eatoms`. The `min_style` specifies that conjugate gradient will be used for minimization and the `minimize` starts the minimization, *i.e.*, the simulation cell boundaries are relaxed from the specified lattice constant (4 Angstroms) to the actual lattice constant (4.05 Angstroms). 

Now we can access some variables to understand the thermodynamic state of the system. First, get the number of atoms in the system-

In [18]:
lmp.natoms

8

We can access the potential energy through the compute we used above, or using the system variable `pe`.

In [20]:
lmp.extract_compute("eatoms", 0, 0), lmp.pe

(-26.879999905465446, array(-26.87999991))

The cohesive energy can then easily be calculated by,

In [22]:
lmp.pe/lmp.natoms

-3.359999988183184

More information about the calculation is also available in `log.lammps`, this can be examined by,

In [23]:
!cat log.lammps

LAMMPS (3 Mar 2020)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:94)
  using 1 OpenMP thread(s) per MPI task
units metal
dimension 3
boundary p p p
atom_style atomic
lattice fcc 4 orient x 1 1 0 orient y -1 1 0 orient z 0 0 1
Lattice spacing in x,y,z = 5.65685 5.65685 4
region box block 0 1 0 1 0 1 units lattice
create_box 1 box
Created orthogonal box = (0 0 0) to (5.65685 5.65685 4)
  1 by 1 by 1 MPI processor grid
create_atoms 1 box
Created 8 atoms
  create_atoms CPU = 0.000788623 secs
replicate 1 1 1
  orthogonal box = (0 0 0) to (5.65685 5.65685 4)
  1 by 1 by 1 MPI processor grid
  8 atoms
  replicate CPU = 0.000645911 secs
pair_style eam/alloy
pair_coeff  * * Al99.eam.alloy Al
neighbor 2.0 bin
neigh_modify delay 10 check yes
compute eng all pe/atom
compute eatoms all reduce sum c_eng
reset_timestep 0
fix 1 all box/relax iso 0.0 vmax 0.001
thermo 10
thermo_style custom step pe lx ly lz press c_eatoms
min_style cg
minimize 1e-05 1e-05 5000 10000
Nei

***
## FAQs 

<br>
<div class="alert alert-danger">
<strong>Question 1</strong>: But I wanted a simulation cell with <110> directions.  How would I change this?
</div>

Aha!  Yes, it is relatively easy to start changing the directions in the x, y, and z directions.  For instance, if you change this line: <br>
`lattice fcc 4 orient x 1 0 0 orient y 0 1 0 orient z 0 0 1`

to this one: <br>
`lattice fcc 4 orient x 1 1 0 orient y -1 1 0 orient z 0 0 1`

then you get a simulation cell that is oriented in the $<110>$ directions in the x- and y- directions.  If you mess up on the math, i.e., try 110 and 110 for the x and y directions (without the '-1'), then LAMMPS will return: <br>
`ERROR: Lattice orient vectors are not orthogonal`

So, if you try the $<110>$-oriented single crystal cell, LAMMPS automatically scales the x- and y-direction periodic lengths to 5.6568542 Angstroms (i.e., $4\sqrt{2}$).  After minimization, the cohesive energy per atom is `-3.35999998818379` ev, th
    
<br>
<div class="alert alert-danger">
<strong>Question 2</strong>: I typed in the above line and nothing happened.
</div>

Make sure that you are in the same directory as the LAMMPS executable. Make sure that you are typing in the correct filename. 

<br>
<div class="alert alert-danger">
<strong>Question 3</strong>: I keep getting an error with the potential, i.e., potential file not found.
</div>

First, check that the potential file is in the directory that you are running out of. Although, you can insert paths, if you want to store the potentials in another directory. Second, make sure that the potential file name is the same as that given in the input script. For instance, Windows has the habit of saving text files (like the potential files) with .txt extensions. If you happen to be running from a Windows operating system, I would change the settings so that you can see the extensions of the files as well. 

***
## Tutorial Links

[Click here to open the next tutorial](LAMMPS-Tutorials-02.ipynb)